In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')


基本変数定義

In [2]:
sampling_flg=0 #サンプリング有無をコントロール
submit_flg=1 #保存するかをコントロール（サンプリングしない時のみ）

SEED=12345
sample_num=10000
fold_num=5

#train関連
train_dir='../../01_input/train.csv'
train_mean_dir='../../02_feature/122-2_train_mean.csv'
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_dir='../../01_input/test.csv'
test_mean_dir='../../02_feature/122-2_test_mean.csv'
test_drop_col=['ID_code']

#予測結果
pred_test_dir='../../04_predict_test/oka_252-2_LightGBM_submission.csv'
pred_train_dir='../../03_predict_train/oka_252-2_LightGBM_train.csv'

#結果ファイル関連　nameは自分の名前に変更する
train_preds_dir='../../03_predict_train/name_253_LightGBM_train.csv'
test_preds_dir='../../04_predict_test/name_253_LightGBM_submission.csv'
save_col_name='oof_xgb'

sample_submission_dir='../../01_input/sample_submission.csv'
submission_target_col_name='target'
submission_id_col_name='ID_code'

前処理

In [3]:
#ロード
train_df=pd.read_csv(train_dir)
test_df=pd.read_csv(test_dir)
train_mean_df=pd.read_csv(train_mean_dir,header=None,index_col=0)
test_mean_df=pd.read_csv(test_mean_dir,header=None,index_col=0)

In [4]:
train_df['mean']=train_mean_df

In [5]:
len(train_df[~((train_df['mean']>0.5) & (train_df['target']==0))])

188945

In [6]:
#train_df=train_df[~((train_df['mean']>0.5) & (train_df['target']==0))]

In [7]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

モデル実行

In [8]:
#model_param
param = {
    "objective" : "binary", 
    "boost":"gbdt",
    "metric":"auc",
    "boost_from_average":"false",
    "num_threads":28,
    "learning_rate" : 0.01,
    "num_leaves" : 13,
    "max_depth":-1,
    "tree_learner" : "serial",
    "feature_fraction" : 0.05,
    "bagging_freq" : 5,
    "bagging_fraction" : 0.4,
    "min_data_in_leaf" : 80,
    "min_sum_hessian_in_leaf" : 10.0,
    "verbosity" : 1,
    'seed': 44000,
    }

In [ ]:
folds = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=SEED)
oof_preds = np.zeros((len(x_train), 1))
test_preds = np.zeros((len(x_test), 1))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(x_train.values, y_train.values)):
    print("\n")
    print("Fold {}".format(fold_))
    
    train_df_NoiseDel=train_df.iloc[trn_idx]
    print(len(train_df_NoiseDel))
    train_df_NoiseDel=train_df_NoiseDel[~((train_df['mean']>0.5) & (train_df['target']==0))]
    print(len(train_df_NoiseDel))
    trn_x=train_df_NoiseDel.drop(train_drop_col,axis=1)
    trn_y=train_df_NoiseDel[train_label]

#    trn_x,trn_y = x_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x,val_y = x_train.iloc[val_idx], y_train.iloc[val_idx]
    trn_data = lgb.Dataset(trn_x,trn_y)
    val_data = lgb.Dataset(val_x,val_y)
    
    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    val_pred = clf.predict(val_x, num_iteration=clf.best_iteration)
    test_pred = clf.predict(x_test, num_iteration=clf.best_iteration)
    
    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    #print("val = {}".format(val_pred)
    oof_preds[val_idx, :] = val_pred.reshape((-1, 1))
    test_preds += test_pred.reshape((-1, 1))
    
    print(confusion_matrix(val_y, pd.DataFrame(np.round(val_pred))))
    
test_preds /= fold_num
roc_score = roc_auc_score(y_train, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))
print(confusion_matrix(y_train, pd.DataFrame(np.round(oof_preds))))



Fold 0
159999
151156
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.934018	valid_1's auc: 0.893053
[2000]	training's auc: 0.941214	valid_1's auc: 0.897205
[3000]	training's auc: 0.947818	valid_1's auc: 0.89835
[4000]	training's auc: 0.953844	valid_1's auc: 0.898459
[5000]	training's auc: 0.959638	valid_1's auc: 0.898201
[6000]	training's auc: 0.964812	valid_1's auc: 0.897635
Early stopping, best iteration is:
[3979]	training's auc: 0.953718	valid_1's auc: 0.89848
AUC = 0.8984802094969693
[[33749  2232]
 [ 1499  2521]]


Fold 1
159999
151125
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.934081	valid_1's auc: 0.892482
[2000]	training's auc: 0.941338	valid_1's auc: 0.89631
[3000]	training's auc: 0.948111	valid_1's auc: 0.897489
[4000]	training's auc: 0.954208	valid_1's auc: 0.897876
[5000]	training's auc: 0.959961	valid_1's auc: 0.897656
[6000]	training's auc: 0.965436	valid_1's auc: 0.897371
[7000]	tra

In [ ]:
#結果保存
if (submit_flg ==1 and sampling_flg==0):
    series_oof_preds = pd.Series(data=oof_preds[:,0], name=save_col_name, dtype='float')
    series_oof_preds.to_csv(train_preds_dir,header=True, index=False)

    sample = pd.read_csv(sample_submission_dir)
    sample.target = test_preds[:,0].astype(float)
    sample.ID_code = test_df['ID_code']
    sample.to_csv(test_preds_dir, index=False)
    